In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# read in all the words
words = open('../../names.txt', 'r').read().splitlines()

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

# import random
# random.seed(42)
# random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182778, 3]) torch.Size([182778])
torch.Size([22633, 3]) torch.Size([22633])
torch.Size([22735, 3]) torch.Size([22735])


In [7]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 5), generator=g)
W1 = torch.randn((15, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, W2]
for p in parameters:
  p.requires_grad = True


In [8]:
lri = []
lossi = []
stepi = []

In [9]:
Xtr

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [ 1, 13,  9],
        [13,  9, 18],
        [ 9, 18, 18]])

In [10]:
import numpy as np

# List to store weight changes for visualization
weight_changes = []

for i in range(1000):
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (32,))
  # forward pass
  emb = C[Xtr] # (32, 3, 10)
  h = torch.tanh(emb.view(-1, 15) @ W1) # (32, 200)
  logits = h @ W2  # (32, 27)
  loss = F.cross_entropy(logits, Ytr)
  #print(loss.item())
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  #lr = lrs[i]
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad


  # track stats
  #lri.append(lre[i])
  # stepi.append(i)
  # lossi.append(loss.log10().item())

In [13]:
%reload_ext tensorboard

In [15]:
# Assuming you've already loaded your Jupyter notebook in VSCode and are ready to go

# Import necessary libraries
import torch
from torch.utils.tensorboard import SummaryWriter

# Load TensorBoard magic command in Jupyter

# Create dummy embeddings and metadata for demonstration
# Replace this with your actual embeddings if you have them
#metadata = [f'Point {i}' for i in range(182625)]  # Just using a generic label here for demonstration
metadata = ["".join([itos[idx] for idx in seq]) for seq in Xtr.tolist()]

# Set up TensorBoard writer and add embeddings
writer = SummaryWriter('runs/embeddings_demo')

# Register the embeddings and the metadata
writer.add_embedding(C[Xtr].view(-1,15), metadata=metadata, tag='my_30d_embeddings')

writer.close()

# Start TensorBoard within the notebook
%tensorboard --logdir runs/embeddings_demo_6


In [ ]:
# Assuming you've already loaded your Jupyter notebook in VSCode and are ready to go

# Import necessary libraries
import torch
from torch.utils.tensorboard import SummaryWriter

# Load TensorBoard magic command in Jupyter

# Create dummy embeddings and metadata for demonstration
# Replace this with your actual embeddings if you have them
#metadata = [f'Point {i}' for i in range(182625)]  # Just using a generic label here for demonstration
metadata = ["".join([itos[idx] for idx in seq]) for seq in Xtr.tolist()]

# Set up TensorBoard writer and add embeddings
writer = SummaryWriter('runs/embeddings_demo_4')

# Register the embeddings and the metadata
writer.add_embedding(C1[Xtr].view(-1,30), metadata=metadata, tag='my_30d_embeddings_4')

writer.close()

# Start TensorBoard within the notebook
%tensorboard --logdir runs/embeddings_demo_4


In [ ]:
Xtr


In [ ]:
str(stoi)


In [ ]:
emb = C[Xtr] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ytr)
loss

In [ ]:
emb = C[Xdev] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ydev)
loss

In [ ]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))